In [1]:
import requests
import msal
from dotenv import load_dotenv
import os

# Load biến môi trường
load_dotenv()

# Azure Client ID, Tenant ID, Client Secret từ file .env
azure_client_id = os.getenv("AZURE_CLIENT_ID")
azure_tenant_id = os.getenv("AZURE_TENANT_ID")
azure_client_secret = os.getenv("AZURE_CLIENT_SECRET")

# SharePoint Site URL và ID của Drive
sharepoint_site_url = "maithujsc.sharepoint.com/sites/Trainingdocument"
drive_id = "b!SJpkxkt_aECkl7ZK6YMWBTM-60BFIl5ChlC_cxyDngG7XD9-vWJITZvMeqzfYkAW"

In [15]:

def get_access_token():
    """Lấy token truy cập Microsoft Graph API"""
    app = msal.ConfidentialClientApplication(
        azure_client_id,
        authority=f"https://login.microsoftonline.com/{azure_tenant_id}",
        client_credential=azure_client_secret
    )
    token = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    return token["access_token"]

In [16]:
def get_files_in_folder():
    """Lấy danh sách các file PDF trong thư mục"""
    token = get_access_token()
    headers = {"Authorization": f"Bearer {token}"}
    
    # URL để lấy các file trong thư mục gốc của drive
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
    
    response = requests.get(url, headers=headers)
    
    # In thêm thông tin debug
    print(f"Response status code: {response.status_code}")
    if response.status_code == 200:
        files = response.json().get("value", [])
        pdf_files = [file["name"] for file in files if file["name"].endswith(".pdf")]
        print(f"📂 Tìm thấy {len(pdf_files)} file PDF:", pdf_files)
        return pdf_files
    else:
        print("❌ Lỗi lấy danh sách file:", response.json())
        return []

In [17]:
def download_file(file_name):
    """Tải file PDF từ SharePoint"""
    token = get_access_token()
    headers = {"Authorization": f"Bearer {token}"}
    url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{file_name}:/content"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Lưu file vào thư mục downloads
        with open(f"downloads/{file_name}", "wb") as f:
            f.write(response.content)
        print(f"✅ Đã tải file: {file_name}")
    else:
        print(f"❌ Lỗi tải file {file_name}: {response.json()}")

# Tạo thư mục "downloads" nếu chưa có
os.makedirs("downloads", exist_ok=True)

# Lấy danh sách file PDF và tải xuống
pdf_files = get_files_in_folder()
for file in pdf_files:
    download_file(file)

Response status code: 200
📂 Tìm thấy 5 file PDF: ['01. Mai Thu Packaging.pdf', 'Dao tao van hoa hoi nhap Mai Thu.pdf', 'Hướng dẫn sử dụng bộ app quản lý.pdf', 'HƯỚNG DẪN TẠO CHỮ KÝ EMAIL MỚI (1).pdf', 'Quản lý đơn hàng-phiếu SX - Power Apps.pdf']
✅ Đã tải file: 01. Mai Thu Packaging.pdf
✅ Đã tải file: Dao tao van hoa hoi nhap Mai Thu.pdf
✅ Đã tải file: Hướng dẫn sử dụng bộ app quản lý.pdf
✅ Đã tải file: HƯỚNG DẪN TẠO CHỮ KÝ EMAIL MỚI (1).pdf
✅ Đã tải file: Quản lý đơn hàng-phiếu SX - Power Apps.pdf


In [5]:
import fitz  # PyMuPDF
import openai
import json
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

def extract_text_from_pdf(pdf_file):
    """Trích xuất văn bản từ file PDF"""
    doc = fitz.open(pdf_file)  # Mở file PDF
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()  # Trích xuất văn bản từ mỗi trang
    return text



# Tạo embedding từ văn bản
def get_embedding(text):
    response = openai.Embedding.create(model="text-embedding-ada-002", input=text)
    embedding = response["data"][0]["embedding"]

    return response.data[0].embedding


In [6]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Tạo thư mục lưu trữ cơ sở dữ liệu
collection = chroma_client.get_or_create_collection(name="training_docs")  # Tạo collection lưu tài liệu đào tạo

for file_name in pdf_files:
    pdf_path = f"downloads/{file_name}"
    text = extract_text_from_pdf(pdf_path)

    if text.strip():  # Kiểm tra xem file có nội dung không
        collection.add(
            documents=[text],
            embeddings=[get_embedding(text)],
            ids=[file_name]
        )
        print(f"✅ Đã lưu vào ChromaDB: {file_name}")
    else:
        print(f"⚠️ File {file_name} không có nội dung!")
        
# Kiểm tra số lượng tài liệu trong ChromaDB
print(f"📝 Số lượng tài liệu trong ChromaDB: {collection.count()}")



NameError: name 'pdf_files' is not defined

In [ ]:
def update_documents_periodically():
    """Kiểm tra và tải tài liệu mới định kỳ"""
    while True:
        print("Đang kiểm tra tài liệu mới từ SharePoint...")
        pdf_files = get_files_in_folder() 
        for file in pdf_files:
            if file not in downloaded_files:
                download_file(file) 
                downloaded_files.append(file)
        time.sleep(7200)  

In [ ]:
def update_chromadb_with_new_documents(pdf_files):
    """Cập nhật tài liệu mới vào ChromaDB"""
    for file_name in pdf_files:
        pdf_path = f"downloads/{file_name}"
        text = extract_text_from_pdf(pdf_path) 

        if text.strip():  
            collection.add(
                documents=[text],
                embeddings=[get_embedding(text)],
                ids=[file_name]
            )
            print(f"✅ Đã cập nhật vào ChromaDB: {file_name}")
        else:
            print(f"⚠️ File {file_name} không có nội dung!")

In [12]:
def collect_user_feedback(query, answer):
    """Thu thập phản hồi từ người dùng và lưu lại vào file"""
    feedback = input("Câu trả lời này có chính xác không? (yes/no): ").lower()
    
    if feedback in ["yes", "y"]:
        print("Cảm ơn bạn!")
        # Lưu lại phản hồi chính xác nếu cần (nếu bạn muốn lưu)
        return True
    else:
        print("Cảm ơn bạn đã phản hồi! Tôi sẽ cải thiện.")
        # Lưu lại các câu trả lời không chính xác để phân tích
        with open("feedback_log.txt", "a", encoding="utf-8") as f:
            f.write(f"Question: {query}, Answer: {answer}, Feedback: Incorrect\n")
        return False

In [7]:
def search_in_chroma(query, top_k=3):
    embedding = get_embedding(query)
    results = collection.query(
        query_embeddings=[embedding],
        n_results=top_k
    )
    
    # Trả về danh sách các văn bản, không phải danh sách con
    return [result[0] for result in results['documents']] 

In [9]:
# Trả lời câu hỏi dựa trên văn bản từ ChromaDB
def generate_answer(query):
    context = "\n".join(search_in_chroma(query))  # Kết hợp các đoạn văn bản thành một chuỗi
    if context:
        prompt = f"Trả lời câu hỏi sau dựa trên thông tin dưới đây:\n\n{context}\n\nCâu hỏi: {query}\nTrả lời:"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Chọn mô hình gpt-3.5-turbo thay vì text-davinci-003
            messages=[
                {"role": "system", "content": "Bạn là một trợ lý thông minh."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )
        return response['choices'][0]['message']['content'].strip()
    else:
        return "Xin lỗi, tôi không thể tìm thấy thông tin liên quan đến câu hỏi của bạn."


In [10]:
def chat():
    print("Chào bạn! Tôi là trợ lý ảo của Mai Thư. Bạn có thể hỏi tôi bất cứ câu hỏi nào.")
    while True:
        user_input = input("Bạn: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Tạm biệt!")
            break

        print(f"\nCâu hỏi của bạn: {user_input}\n")
        answer = generate_answer(user_input) 
        
        print(f"Chatbot: {answer}")
        
        collect_user_feedback(user_input, answer)
        
        

In [13]:
if __name__ == "__main__":
    chat()


Chào bạn! Tôi là trợ lý ảo của Mai Thư. Bạn có thể hỏi tôi bất cứ câu hỏi nào.

Câu hỏi của bạn: i need some informations of Take away box

Chatbot: Thông tin về hộp đựng thức ăn mang về (Take away box) bao gồm:
- Vật liệu: Kraft, Ivory, giấy carton 3 lớp,...
- Trọng lượng giấy: 250gsm - 400gsm,...
- Phương pháp in: Offset / Flexo, 1 màu / nhiều màu
- Kiểu dáng: Có cửa sổ / không có cửa sổ
- Các kích thước và loại hộp khác nhau như hộp cốc coffee mang về, hộ
Cảm ơn bạn!

Câu hỏi của bạn: what about  Cup carrier tray

Chatbot: Cup carrier tray is one of the products in Mai Thu's product range. It is made from white kraft and brown kraft with a paper weight of 350gsm. The printing method used for Cup carrier tray is Offset/Flexo, and it comes in a style with 2 cups or 4 cups.
Cảm ơn bạn đã phản hồi! Tôi sẽ cải thiện.

Câu hỏi của bạn: and bread bag

Chatbot: Bread bag.
Cảm ơn bạn đã phản hồi! Tôi sẽ cải thiện.

Câu hỏi của bạn: Do mai thu packaging produce toys for kids?

Chatbot: Dựa và